<a href="https://colab.research.google.com/github/kkimtaejung/All_heuristic/blob/GAN/ROBO_FACE_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 얼굴 이미지

## 숫자와는 다른 문제점
1. 컬러 이미지를 훈련하고, 컬러 이미지를 생성해야 한다.

2. 사진의 훈련 데이터셋을 가지고 좀 더 다양하고 그럴듯한 결과를 얻어야 한다.

## 데이터셋 불러오기
- 코드가 시작되고 끝날때까지만 유효한 데이터, 데이터양이 방대하여 빠르게 데이터셋을 이용하고자 h5py file을 이용하였다.

In [ ]:

# mount Drive to access data files

from google.colab import drive
drive.mount('./mount')

In [ ]:
import h5py
import zipfile
import imageio
import os
# location of the HDF5 package, yours may be under /gan/ not /myo_gan/
hdf5_file = 'ROBO.h5py'

# how many of the 202,599 images to extract and package into HDF5
total_images = 20000

with h5py.File(hdf5_file, 'w') as hf:

    count = 0

    with zipfile.ZipFile('/content/mount/MyDrive/ROBOTICS/img_align_celeba.zip', 'r') as zf:
      for i in zf.namelist():
        if (i[-4:] == '.jpg'):
          # extract image
          ofile = zf.extract(i)
          img = imageio.imread(ofile)
          os.remove(ofile)

          # add image data to HDF5 file with new name
          hf.create_dataset('img_align_celeba/'+str(count)+'.jpg', data=img, compression="gzip", compression_opts=9)
          
          count = count + 1
          if (count%1000 == 0):
            print("images done .. ", count)
            pass
            
          # stop when total_images reached
          if (count == total_images):
            break
          pass

        pass
      pass

## 데이터 살펴보기

In [ ]:
# 정말 사람의 이미지를 담고 있을까?
import numpy as np
import matplotlib.pyplot as plt
with h5py.File('/content/ROBO.h5py','r') as file_object:
  dataset = file_object['img_align_celeba']
  image = np.array(dataset['7.jpg'])
  plt.imshow(image, interpolation='none')
  pass

In [ ]:
# 넘파이 행렬로 변환된 이미지의 형태, 색 정보를 확인해보자!
image.shape

높이 218 너비 178 rgb 3

총 3개의 레이어 확인 가능

## 데이터셋 클래스

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import pandas, numpy, random
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

class CelebADataset(Dataset):
  # HDF5 파일을 열고, img_align_celeba로 각각의 이미지에 접근 가능하게 하는 __init__ 생성자
  def __init__(self,file):
    self.file_object = h5py.File(file, 'r')
    self.dataset = self.file_object['img_align_celeba']
    pass

  def __len__(self):
    return len(self.dataset)

  # index를 이미지의 이름으로 변환, 이미지 데이터르 반환하는 기능
  # 정규화도 마쳐서 return
  # if문은 인덱스가 실제 데이터셋의 아이템 수보다 많은지 확인, 많으면 IndexError 예외 발생
  # 예외 발생시키는 이유는, HDF5에서는 범위 벗어나는 요청을 하면 그냥 오류가 발생하기 때문에 예외 처리함
  def __getitem__(self,index):    
    if(index>=len(self.dataset)):
      raise IndexError()
    img = numpy.array(self.dataset[str(index)+'.jpg'])
    return torch.cuda.FloatTensor(img)/255.0

  def plot_image(self, index):
    plt.imshow(numpy.array(self.dataset[str(index)+'.jpg']), interpolation='nearest')
    pass
  pass

In [ ]:
# Dataset 객체 생성
celeba_dataset = CelebADataset('/content/ROBO.h5py')

# 데이터 확인
celeba_dataset.plot_image(43)

## 판별기

MNIST와 다른점은, 색깔과 크기 정보다.

하나의 이미지는 218 x 178 x 3 = 116,412 개의 노드가 필요

116,412 노드를 -> 1차원 텐서로 바꾸고 -> 완전연결 신경망에 투입

::랜덤시드::

In [ ]:
def generate_random_image(size):
  random_data = torch.rand(size)
  return random_data
def generate_random_seed(size):
  random_data = torch.randn(size)
  return random_data

::View 클래스 생성::

3차원 이미지 텐서를 -> 1차원 이미지 텐서로 바꿔주는 역할

In [ ]:
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape,

    def forward(self, x):
        return x.view(*self.shape)

In [ ]:
class Discriminator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        View(218*178*3),
        nn.Linear(218*178*3,100),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(100),
        nn.Linear(100,1),
        nn.Sigmoid())
    # 구조 :: node4 -> node3(로지스틱) -> node1(로지스틱) -> 1 or 0

    # 손실함수 설정
    self.loss_function = nn.BCELoss()

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.01)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, inputs):
    # 모델 실행
    return self.model(inputs)

  # 훈련 과정
  def train(self,inputs,targets):
    #신경망 출력 계산
    outputs = self.forward(inputs)

    #손실 계산
    loss = self.loss_function(outputs,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    if(self.counter%10000==0):
      print("counter = ",self.counter)
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
  
  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5))
    pass

## 판별기 테스트하기

일단 GPU 부터 설정하기

In [ ]:
import torch
import torchvision
!pip install --upgrade torch torchvision

In [ ]:
import torch
import torchvision
if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)
  print("using cuda: ", torch.cuda.get_device_name(0))
  pass
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# 판별기가 실제 데이터와 임의 노이즈 구별이 가능한지
D=Discriminator()
D.to(device)

for image_data_tensor in celeba_dataset:
  # 실제 데이터
  D.train(image_data_tensor, torch.cuda.FloatTensor([1.0]))
  # 생성된 데이터
  D.train(generate_random_image((218,178,3)), torch.cuda.FloatTensor([0.0]))
  pass

In [ ]:
D.plot_progress()

## 생성기

In [ ]:
class Generator(nn.Module):
  def __init__(self):
    # 파이토치 부모 클래스 초기화
    super().__init__()

    # 신경망 레이어 정의
    self.model = nn.Sequential(
        nn.Linear(100,3*10*10),
        nn.LeakyReLU(0.02),
        nn.LayerNorm(3*10*10),
        nn.Linear(3*10*10,3*218*178),
        nn.Sigmoid(),
        View((218,178,3)))
    # 구조 :: node1 -> node3(로지스틱) -> node4(로지스틱)

    # 손실함수 없음 -> 오직 판별기에만 필요
    # 판별기의 흘러들어온 기울기 오차 -> 생성기 업데이트

    # SGD 옵티마이저 설정
    self.optimiser = torch.optim.Adam(self.parameters(), lr = 0.01)

    # 진행 측정을 위한 변수 초기화
    self.counter = 0
    self.progress = []
    pass
  
  # 순전파 과정
  def forward(self, inputs):
    # 모델 실행
    return self.model(inputs)

  # 훈련 과정
  def train(self,D,inputs,targets):
    #신경망 출력 계산
    g_output = self.forward(inputs)

    #판별기로 전달
    d_output = D.forward(g_output)

    #오차 계산
    loss = D.loss_function(d_output,targets)

    #카운터를 증가시키고 10회마다 오차 저장
    self.counter +=1
    if(self.counter%10==0):
      self.progress.append(loss.item())
      pass
    
    #기울기 초기화하고 역전파 후 가중치 갱신
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    pass
# ::train::
# 입력값이 생성기의 forward로 전달됨
# 생성기의 출력 g_output은 D.forward를 통해 판별기로 전달되어 d_output을 출력함
# 손실 = |d_output - 정답| -> 차이값으로 계산
# 손실로부터 오류역전파 되어 생성기로 전달되어 생성기의 optimiser로 전달됨
# 이러면 생성기만 업데이트됨

  # 시각화
  def plot_progress(self):
    df = pandas.DataFrame(self.progress, columns=['loss'])
    df.plot(ylim=(0,1.0), figsize=(16,8), alpha=0.1, marker='.', grid=True, yticks=(0,0.25,0.5))
    pass

## 생성기 결과 확인

In [ ]:
G = Generator()
# 모델을 CUDA에 배치
G.to(device)
output = G.forward(generate_random_seed(100))
img = output.detach().cpu().numpy()
plt.imshow(img, interpolation='none', cmap='Blues')

## GAN 훈련

In [ ]:
D = Discriminator()
D.to(device)
G = Generator()
G.to(device)

epochs = 1
for i in range(epochs):
  for image_data_tensor in celeba_dataset:
    # 1단계 : 참에 대해 판별기 훈련
    D.train(image_data_tensor, torch.cuda.FloatTensor([1.0]))
    # 2단계 : 거짓에 대해 판별기 훈련, 
    # 생성기의 기울기가 수정되지 않도록 detach()함수 이용, 
    # detach()로 선택된 것은 그 함수에서 제외됨
    D.train(G.forward(generate_random_seed(100)).detach(), torch.cuda.FloatTensor([0.0]))
    # 3단계 : 생성기 훈련
    G.train(D, generate_random_seed(100), torch.cuda.FloatTensor([1.0]))

## 결과 시각화

In [ ]:
D.plot_progress()

In [ ]:
G.plot_progress()

In [ ]:
f, axarr = plt.subplots(2,3,figsize=(16,8))
for i in range(2):
  for j in range(3):
    output = G.forward(generate_random_seed(100))
    img = output.detach().cpu().numpy()
    axarr[i,j].imshow(img, interpolation='none', cmap='Blues')
    pass
  pass

# 190pg